In [25]:
import datetime as dt
import os
import pandas as pd
import re
import requests
from datetime import datetime
from pathlib import Path
#os.chdir(r'C:\users\phamj\OneDrive - Jacobs\Desktop')

index_csv_path = Path(r'C:\Users\alden.paroni\unifier-requests\index example - Copy.csv')
disciplines_csv_path = Path(r'C:\Users\alden.paroni\unifier-requests\disciplines.csv')

# Make sure the folder name containing the drawings matches the drawing set name.
files_loc = r'C:\Users\alden.paroni\Downloads\CCUR IV Roadway'


# this function generates a timestamp. returns string with no dashes/colons/periods
def gen_ts():    
    return datetime.now().isoformat().replace('-','').replace(':','').replace('.','')


In [26]:
env = 'prod'
project_number = 'DBB511.2'

Update the display settings in pandas

In [27]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

Load the index into a df

In [28]:
df = pd.read_csv(index_csv_path)

Load the disciplines into a df

In [29]:
disc = pd.read_csv(disciplines_csv_path)

Load the directory contents into a df

In [30]:
files = []
directory_contents = list(Path(files_loc).rglob('*'))
# pdf_paths is the paths to all pdfs in all subfolders
pdf_paths = [p for p in directory_contents if p.suffix=='.pdf']

for p in pdf_paths:
    files.append({'folder': p.parent.name
                , 'file': p.name
                , 'file_path': str(p)
        })
# files = []
# for folder in os.listdir():
#     os.chdir(folder)
#     for file in os.listdir():
#         files.append({
#             'folder': folder,
#             'file': file,
#             'file_path': os.path.abspath(file).replace('\\', '/'),
#             'file_path': os.path.abspath(file)
#         })
#     os.chdir('..')

files = pd.DataFrame(files)

Extract the drawing number from the file

In [31]:
files['drawing_number'] = files['file'].apply(
    lambda s: re.sub(
        pattern='.pdf',
        repl='',
        string=s,
        flags=re.IGNORECASE
    ).strip()
)

Join the index with the files df

In [32]:
df = df.merge(
    right=files.set_index(['folder', 'drawing_number']),
    how='left',
    left_on=['drawing_set', 'drawing_number'],
    right_on=['folder', 'drawing_number']
)

Check if any files were unable to be located in the directory

In [33]:
# the output of this cell is ideally empty
df[df['file'].isna()]

,drawing_set,original_drawing_number,drawing_number,contractor_ref_number,discipline_code,drawing_phase,revision_number,title,file,file_path


Join the files with the index df

In [34]:
temp = files.merge(
    right=df,
    left_on=['folder', 'drawing_number'],
    right_on=['drawing_set', 'drawing_number'],
    how='left'
)

Check if any files were unable to be located in the index

In [35]:
temp[temp['revision_number'].isna()].drop(
    ['file_y', 'file_path_y'],
    axis=1
).rename(
    mapper={'file_x': 'file', 'file_path_x': 'file_path'},
    axis=1
)

,folder,file,file_path,drawing_number,drawing_set,original_drawing_number,contractor_ref_number,discipline_code,drawing_phase,revision_number,title


Add the revised discipline code

In [36]:
df['revised_discipline_code'] = df['discipline_code'].apply(
    lambda s: s[0]
)

Add the discipline

In [37]:
df = df.join(
    other=disc[['code', 'discipline_upper']].set_index(keys=['code']),
    on=['revised_discipline_code'],
    how='left',
    rsuffix='_b'
)

Update the title

In [38]:
df['revised_title'] = df['title'].apply(
    lambda s: s.title()
)

Check if the title is under 120 characters

In [39]:
df[df['revised_title'].str.len() > 120]

,drawing_set,original_drawing_number,drawing_number,contractor_ref_number,discipline_code,drawing_phase,revision_number,title,file,file_path,revised_discipline_code,discipline_upper,revised_title


Add columns

In [40]:
df['project_number'] = project_number
df['safety_impact'] = 'No'
df['om_impact'] = 'No'
df['quality_impact'] = 'No'
df['row_impact'] = 'No'
df['revision_record_no'] = None

Sort the df

In [41]:
df.sort_values(
    by=['drawing_set', 'drawing_number'],
    inplace=True
)

Set up logs

In [42]:
from unifier_requests.ur import write_dicts_to_csv

## Upload the drawing sets (SKIP THIS STEP FOR REVISIONS)


In [21]:
from unifier_requests.bplib import uxdrwset
results = []
x = uxdrwset(env = env, project_number = project_number)
for row in df[['drawing_set', 'project_number']].drop_duplicates().itertuples():
    data = {
        'project_number': row.project_number,
        'title': row.drawing_set
    }        
    res = x.create(data = data)
    # res = req.create_record(
    #     project_number=row.project_number,
    #     bpname='Drawing Sets',
    #     data=data
    # )
    results.append(res)
    if res['status'] == 200:
         print(f'Success: {row.drawing_set}')
    else:
         print(f'Error: {row.drawing_set}')

200 
Success: CCUR IV Roadway


Write the results to the log

In [37]:
#req.write_log(results)

write_dicts_to_csv(input_dicts = results, name = 'uxdrwset_upload'+gen_ts())

WindowsPath('C:/Users/phamj/ureqlogs/uxdrwset_upload20230419T113232337214.csv')

Upload the drawings (SKIP THIS STEP FOR REVISIONS)

In [22]:
from unifier_requests.bplib import uxdrw
results = []
x = uxdrw(env = env, project_number = project_number)
for row in df[[
    'contractor_ref_number',
    'discipline_upper',
    'drawing_number',
    'drawing_set',
    'om_impact',
    'project_number',
    'quality_impact',
    'revised_title',
    'row_impact',
    'safety_impact'
]].itertuples():
    data = {
        'draDrawingSetsDPK': row.drawing_set,
        'draDrawingSetsTB250': row.drawing_set,
        'genCtrRefNoTB100': row.contractor_ref_number,
        'project_number': row.project_number,
        'subpomimpactrb': row.om_impact,
        'subpqualityimpactrb': row.quality_impact,
        'subprowrb': row.row_impact,
        'subpsafetyimpactrb': row.safety_impact,
        'udrDrawingNoTB120': row.drawing_number,
        'udrDrawingTitleTB120': row.revised_title,
        'ugenDiscipline': row.discipline_upper
    }    
    res = x.create(data = data)
    # res = req.create_record(
    #     project_number=row.project_number,
    #     bpname='Drawings',
    #     data=data
    # )
    results.append(res)
    if res['status'] == 200:
        print(f'Success: {row.drawing_set} -> {row.drawing_number}')
    else:
        print(f'Error: {row.drawing_set} -> {row.drawing_number}')

200 
Success: CCUR IV Roadway -> CCUR4-A03-GN0005R
200 
Success: CCUR IV Roadway -> CCUR4-C04-TR5372AR


Write the results to the log

In [149]:
write_dicts_to_csv(input_dicts = results, name = 'uxdrw_upload'+gen_ts())
#req.write_log(results)

WindowsPath('C:/Users/phamj/ureqlogs/uxdrw_upload20230503T144551735154.csv')

Upload the drawing revisions

In [43]:
from unifier_requests.bplib import uxdrwrev
results = []
x = uxdrwrev(env = env, project_number = project_number)
for row in df[[
    'drawing_number',
    'drawing_phase',
    'drawing_set',
    'project_number',
    'revision_number'
]].itertuples():
    data = {
        'project_number': row.project_number,
        'udrDrawingNoTB120': row.drawing_number,
        'udrDrawingPhaseDP': row.drawing_phase,
        'udrawDrawingBP': row.drawing_number,
        'udrawRevNoBP': row.revision_number
    }    
    res = x.create(data)
    # res = req.create_record(
    #     project_number=row.project_number,
    #     bpname='Drawing Revisions',
    #     data=data
    # )
    results.append(res)
    if res['status'] == 200:
        print(f'Success: {row.drawing_set} -> {row.drawing_number}')
    else:
        print(f'Error: {row.drawing_set} -> {row.drawing_number}')
    try:
        df.at[row.Index, 'revision_record_no'] = res['message'][0]['record_no']
    except:
        pass

200 
Success: CCUR IV Roadway -> CCUR4-A03-GN0005R
200 
Success: CCUR IV Roadway -> CCUR4-C04-TR5372AR


Write the results to the log

In [21]:
write_dicts_to_csv(input_dicts = results, name = 'uxdrwrev_upload'+gen_ts())
#req.write_log(results)

WindowsPath('C:/Users/phamj/ureqlogs/uxdrwrev_upload20230510T142422142428.csv')

Upload drawing revision attachments

In [44]:
from unifier_requests.ur_bp_attach import bp_attach
bpid = 'uxdrwrev'
results = []
for row in df[[
    'drawing_number',
    'drawing_set',
    'file_path',
    'project_number',
    'revision_record_no'
]].itertuples():
    
    res = bp_attach(env = env
                , bpid = bpid
                , project_number = row.project_number
                , record_no = row.revision_record_no
                , file_path = row.file_path)
    # res = req.update_record_attachment(
    #     project_number=row.project_number,
    #     bpname='Drawing Revisions',
    #     record_no=row.revision_record_no,
    #     file_name=row.file_path
    # )
    results.append(res)
    if res['status'] == 200:
        print(f'Success: {row.drawing_set} -> {row.drawing_number}')
    else:
        print(f'Error: {row.drawing_set} -> {row.drawing_number}')
 

200 
Success: CCUR IV Roadway -> CCUR4-A03-GN0005R
200 
Success: CCUR IV Roadway -> CCUR4-C04-TR5372AR


Write the results to the log

In [23]:
write_dicts_to_csv(input_dicts = results, name = f'drawing_revisions_upper_form_drawing_revision_attachments_{gen_ts()}')
#req.write_log(results)

WindowsPath('C:/Users/phamj/ureqlogs/drawing_revisions_upper_form_drawing_revision_attachments_20230510T143724275381.csv')